In [1]:
from networks.models import build_model

In [2]:
import torch
import cv2
import matplotlib.pyplot as plt

In [3]:
model = build_model("FBA.pth").cuda().eval()

modifying input layer to accept 11 channels


In [4]:
image = torch.randn((1, 3, 128, 128)).cuda()
two_chan_trimap=  torch.randn((1, 2, 128, 128)).cuda()
image_n=torch.randn((1, 3, 128, 128)).cuda()
trimap_transformed =torch.randn((1, 6, 128, 128)).cuda()
output = model(image, two_chan_trimap, image_n, trimap_transformed)

/home/marco/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:693: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch._C._nn.max_pool2d_with_indices(input, kernel_size, stride, padding, dilation, ceil_mode)


In [5]:
output = model(image, two_chan_trimap, image_n, trimap_transformed)

In [6]:
jitmodel = torch.jit.trace(model, [image, two_chan_trimap, image_n, trimap_transformed])


/home/marco/anaconda3/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [7]:
output_jit = jitmodel(image, two_chan_trimap, image_n, trimap_transformed)

In [8]:
print(torch.mean( (output - output_jit)**2).item())

0.0


In [9]:
%timeit _ = jitmodel(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

5.73 ms ± 96.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit _ = model(image, two_chan_trimap, image_n, trimap_transformed); torch.cuda.synchronize()

7.72 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
